In [55]:
import facebook
import json
import pprint
import csv

In [56]:
# Facebook access token hardcoded. Get this from Facebook Graph Explorer
ACCESS_TOKEN = "EAACEdEose0cBAN4VZAjRBZBdxUEnmwSDMSZBbnZBbWAD8Erclx7ZBDGAGHKjhjytVLSSvlk6MWfYQAqH1Iw85daT4U857a8RFFgFz41con0D1IHM4ZA5Xf8ipElRTasRfTqon0ogXQZBE1iPGJGTpCfKkKCrJeI1w1XGxUkGnx3xQZDZD"

In [57]:
# Making API call to get all posts for given ID (in this case, politician's page)
graph = facebook.GraphAPI(access_token=ACCESS_TOKEN, version='2.7')
all_posts = graph.get_object(id='leehsienloong/posts')

In [58]:
# Creating a list of post IDs from the JSON data (returned from API call)  
post_id_list= []

for post in all_posts["data"]:
    post_id_list.append(post["id"])

In [64]:
# Loop through each post, retrieve relevant post information (message, likes count, reactions count, comments count)
# Store data in list. Each element is still in JSON format.
post_desc_list = []
fields_required = "message, shares, comments.limit(0).summary(true), likes.limit(0).summary(true),reactions.type(LOVE).limit(0).summary(total_count).as(reactions_love),reactions.type(WOW).limit(0).summary(total_count).as(reactions_wow),reactions.type(HAHA).limit(0).summary(total_count).as(reactions_haha),reactions.type(SAD).limit(0).summary(total_count).as(reactions_sad),reactions.type(ANGRY).limit(0).summary(total_count).as(reactions_angry)"

for ids in post_id_list:
    post_desc_list.append(graph.get_object(id=ids, fields=fields_required))
    
print len(post_desc_list)
pprint.pprint(post_desc_list[3])

25
{u'comments': {u'data': [],
               u'summary': {u'can_comment': True,
                            u'order': u'ranked',
                            u'total_count': 350}},
 u'id': u'125845680811480_1347625838633452',
 u'likes': {u'data': [],
            u'summary': {u'can_like': True,
                         u'has_liked': False,
                         u'total_count': 7836}},
 u'message': u'For everyone celebrating #ChineseNewYear, I hope you are all prepared! Tonight, Chinese families will be gathering for their reunion dinner. If you can\u2019t make it tonight because of work or other reasons, I hope you have arranged to celebrate it earlier, or hold it on another night. The reunion dinner is an ancient tradition, reminding us of what is truly important in life as we enter into the New Year \u2013 family, loved ones and friends. \n\nThe lunar new year is also a time to reflect on what others have done for us and how we can pay it forward for our future generations. Find ou

In [67]:
# Loop through post_desc_list, Each interation, retrieve relevant info, create row and write into csv file

headers = ["No.","status","comments_count","shares_count","likes_count","angry_count","haha_count","love_count","sad_count","wow_count"]

with open('datafile.csv', 'wb') as f:
    w = csv.writer(f, delimiter=",")
    w.writerow(headers)
    count = 1
    for post_desc in post_desc_list:
        
        try: 
            status_str = str(post_desc["message"].encode('utf8'))
        except KeyError:
            status_str = ""
        
        comments_count = str(post_desc["comments"]["summary"]["total_count"])
        
        try:
            shares_count = str(post_desc["shares"]["count"])
        except KeyError:
            shares_count = 0
        
        likes_count = str(post_desc["likes"]["summary"]["total_count"])
        angry_count = str(post_desc["reactions_angry"]["summary"]["total_count"])
        haha_count = str(post_desc["reactions_haha"]["summary"]["total_count"])
        love_count = str(post_desc["reactions_love"]["summary"]["total_count"])
        sad_count = str(post_desc["reactions_sad"]["summary"]["total_count"])
        wow_count = str(post_desc["reactions_wow"]["summary"]["total_count"])
        
        row_string = [count,status_str,comments_count,shares_count,likes_count,angry_count,haha_count,love_count,sad_count,wow_count]
        w.writerow(row_string)
        count += 1
print count-1,"done"

26 done


In [55]:
data_dict = {}

for item in post_desc_list:
    data_dict["message"] = item["message"]
    data_dict["comments"] = str(item["comments"]["summary"]["total_count"])
    data_dict["likes"] = str(item["likes"]["summary"]["total_count"])
    data_dict["angry"] = str(item["reactions_angry"]["summary"]["total_count"])
    data_dict["haha"] = str(item["reactions_haha"]["summary"]["total_count"])
    data_dict["love"] = str(item["reactions_love"]["summary"]["total_count"])
    data_dict["sad"] = str(item["reactions_sad"]["summary"]["total_count"])
    data_dict["wow"] = str(item["reactions_wow"]["summary"]["total_count"])

#print data_dict

import csv

#new_dict = dict((k.encode('ascii'), v) for (k, v) in data_dict.items())
#print new_dict
with open('datafile.csv', 'wb') as f:
    w = csv.DictWriter(f, data_dict.keys())
    w.writeheader()
    w.writerow({k:v.encode('utf8') for k,v in data_dict.items()})
    
print "done"

# print "Status:", post_desc_list[0]["message"]
# print "Comments:", post_desc_list[0]["comments"]["summary"]["total_count"]
# print "Likes:", post_desc_list[0]["likes"]["summary"]["total_count"]
# print "Reactions:"
# print "- angry", post_desc_list[0]["reactions_angry"]["summary"]["total_count"]
# print "- haha", post_desc_list[0]["reactions_haha"]["summary"]["total_count"]
# print "- love", post_desc_list[0]["reactions_love"]["summary"]["total_count"]
# print "- sad", post_desc_list[0]["reactions_sad"]["summary"]["total_count"]
# print "- wow", post_desc_list[0]["reactions_wow"]["summary"]["total_count"]

AttributeError: 'list' object has no attribute 'encode'